In [1]:
from bs4 import BeautifulSoup
import urllib.request
import string
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity 
import numpy as np
import pickle
import itertools
import pandas as pd

In [2]:
# scrap dari digilib

In [2]:
with open('paper_ori.pkl', 'rb') as f:
    paper = pickle.load(f)

In [3]:
len(paper)

360

In [10]:
# preprocessing

In [10]:
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()
stemmer = StemmerFactory().create_stemmer()

In [4]:
with open('processed_paper.pkl', 'rb') as f:
    processed_paper = pickle.load(f)

In [15]:
# generate thesaurus

In [5]:
with open('thesaurus.pkl', 'rb') as f:
    thesaurus = pickle.load(f)

In [22]:
words = [x for x in thesaurus]

In [81]:
# input query

In [7]:
init_query = 'pengembangan aplikasi'

In [8]:
# tf_idf

In [11]:
vectorizer = TfidfVectorizer(use_idf=True)
query = init_query
query = query.lower()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
query = query.translate(remove_punctuation_map)
query = stopword.remove(query)
query = query.split()
query = [stemmer.stem(x) for x in query]
query

['kembang', 'aplikasi']

In [12]:
max_result = []
x = [' '.join(query)]
paper_tfidf = vectorizer.fit_transform(x + processed_paper)
q = paper_tfidf[0]
result = cosine_similarity(paper_tfidf, q)
idx = np.argsort(-result,axis=0).flatten()    
final = [[num, y[0], x] for num, y in enumerate(result) if y[0] > 0.0]
max_result += final
max_result = sorted(max_result, key=lambda x: x[1], reverse=True)
set_result = set()
new_result = []
for item in max_result:
    if item[0] not in set_result:
        set_result.add(item[0])
        new_result.append(item)
    else:
        pass
new_result

[[0, 1.0, ['kembang aplikasi']],
 [1, 0.31747033786806195, ['kembang aplikasi']],
 [2, 0.31747033786806195, ['kembang aplikasi']],
 [19, 0.31747033786806195, ['kembang aplikasi']],
 [20, 0.31747033786806195, ['kembang aplikasi']],
 [37, 0.31747033786806195, ['kembang aplikasi']],
 [38, 0.31747033786806195, ['kembang aplikasi']],
 [55, 0.31747033786806195, ['kembang aplikasi']],
 [56, 0.31747033786806195, ['kembang aplikasi']],
 [73, 0.31747033786806195, ['kembang aplikasi']],
 [74, 0.31747033786806195, ['kembang aplikasi']],
 [91, 0.31747033786806195, ['kembang aplikasi']],
 [92, 0.31747033786806195, ['kembang aplikasi']],
 [109, 0.31747033786806195, ['kembang aplikasi']],
 [110, 0.31747033786806195, ['kembang aplikasi']],
 [127, 0.31747033786806195, ['kembang aplikasi']],
 [128, 0.31747033786806195, ['kembang aplikasi']],
 [145, 0.31747033786806195, ['kembang aplikasi']],
 [146, 0.31747033786806195, ['kembang aplikasi']],
 [163, 0.31747033786806195, ['kembang aplikasi']],
 [164, 0.317

In [13]:
for x in new_result[1:5]: 
    print('Result', x[0]) 
    print('QUERY', x[2]) 
    print(paper[x[0]-1][1]) 
    print(paper[x[0]-1][2][:200] + '...')
    print()

Result 1
QUERY ['kembang aplikasi']
RANCANG BANGUN APLIKASI KOMUNIKASI AUDIO PADA JARINGAN NIRKABEL LOKAL BERBASIS ANDROID MENGGUNAKAN ALGORITMA JOINT CODING RATE CONTROL
Perkembangan aplikasi Android dewasa ini sangat pesat aplikasi dikembangkan untuk menyelesaikan permasalahan diberbagai bidang. Saat ini ada beberapa kategori aplikasi Android yang dikembangkan sepert...

Result 2
QUERY ['kembang aplikasi']
DESIGN AND IMPLEMENTATION OF ANDROID BASED AUDIO COMMUNICATION OVER WIRELESS LOCAL NETWORK USING JOINT CODING RATE CONTROL ALGORITHM
Perkembangan aplikasi Android dewasa ini sangat pesat aplikasi dikembangkan untuk menyelesaikan permasalahan diberbagai bidang. Saat ini ada beberapa kategori aplikasi Android yang dikembangkan sepert...

Result 19
QUERY ['kembang aplikasi']
RANCANG BANGUN APLIKASI KOMUNIKASI AUDIO PADA JARINGAN NIRKABEL LOKAL BERBASIS ANDROID MENGGUNAKAN ALGORITMA JOINT CODING RATE CONTROL
Perkembangan aplikasi Android dewasa ini sangat pesat aplikasi dikembangkan un

In [17]:
idx_res = [x[0]-1 for x in new_result[1:]]
file = []
for i,x in enumerate(paper):
    if i in idx_res:
        file.append([x[1],x[2],'ok_ok'])
    else:
        file.append([x[1],x[2],''])
df = pd.DataFrame(file)
df.to_excel('hasil ori ' +init_query+ '.xlsx', header=False, index=False)

In [18]:
# query_expansion

In [19]:
vectorizer = TfidfVectorizer(use_idf=True)
query = init_query
query = query.lower()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
query = query.translate(remove_punctuation_map)
query = stopword.remove(query)
query = query.split()
query = [stemmer.stem(x) for x in query]
query

['kembang', 'aplikasi']

In [23]:
product_query = []
list_synonym = []
for x in query:
    if x in words:
        list_synonym.append(thesaurus[x])
    else:
        name = x
        data = { "q": name }
        encoded_data = urllib.parse.urlencode(data).encode("utf-8")
        content = urllib.request.urlopen("http://www.sinonimkata.com/search.php", encoded_data)
        soup = BeautifulSoup(content, 'html.parser')
        try:
            synonym = soup.find('td', attrs={'width': '90%'}).find_all('a')
            synonym = [x.getText() for x in synonym]
            thesaurus[x] = [x] + synonym
            list_synonym.append(thesaurus[x])
        except:
            list_synonym.append([x])
qs = []
for x in itertools.product(*list_synonym):
    x = [stemmer.stem(y) for y in x]
    qs.append([' '.join(x)])
for x in qs:
    print(x, end=' ')

['kembang aplikasi'] ['kembang operasi'] ['kembang laksana'] ['kembang terap'] ['kembang guna'] ['kembang praktik'] ['kembang rekayasa'] ['kembang lamar'] ['kembang minta'] ['kembang mohon'] ['bunga aplikasi'] ['bunga operasi'] ['bunga laksana'] ['bunga terap'] ['bunga guna'] ['bunga praktik'] ['bunga rekayasa'] ['bunga lamar'] ['bunga minta'] ['bunga mohon'] ['kesuma aplikasi'] ['kesuma operasi'] ['kesuma laksana'] ['kesuma terap'] ['kesuma guna'] ['kesuma praktik'] ['kesuma rekayasa'] ['kesuma lamar'] ['kesuma minta'] ['kesuma mohon'] ['kusuma aplikasi'] ['kusuma operasi'] ['kusuma laksana'] ['kusuma terap'] ['kusuma guna'] ['kusuma praktik'] ['kusuma rekayasa'] ['kusuma lamar'] ['kusuma minta'] ['kusuma mohon'] ['puspa aplikasi'] ['puspa operasi'] ['puspa laksana'] ['puspa terap'] ['puspa guna'] ['puspa praktik'] ['puspa rekayasa'] ['puspa lamar'] ['puspa minta'] ['puspa mohon'] ['puspita aplikasi'] ['puspita operasi'] ['puspita laksana'] ['puspita terap'] ['puspita guna'] ['puspita

In [24]:
max_result = []
for x in qs:
    paper_tfidf = vectorizer.fit_transform(x + processed_paper)
    q = paper_tfidf[0]
    result = cosine_similarity(paper_tfidf, q)
    idx = np.argsort(-result,axis=0).flatten()    
    final = [[num, y[0], x] for num, y in enumerate(result) if y[0] > 0.0]
    max_result += final
max_result = sorted(max_result, key=lambda x: x[1], reverse=True)
set_result = set()
new_result = []
for item in max_result:
    if item[0] not in set_result:
        set_result.add(item[0])
        new_result.append(item)
    else:
        pass
new_result

[[0, 1.0000000000000002, ['kembang terap']],
 [1, 0.31747033786806195, ['kembang aplikasi']],
 [2, 0.31747033786806195, ['kembang aplikasi']],
 [19, 0.31747033786806195, ['kembang aplikasi']],
 [20, 0.31747033786806195, ['kembang aplikasi']],
 [37, 0.31747033786806195, ['kembang aplikasi']],
 [38, 0.31747033786806195, ['kembang aplikasi']],
 [55, 0.31747033786806195, ['kembang aplikasi']],
 [56, 0.31747033786806195, ['kembang aplikasi']],
 [73, 0.31747033786806195, ['kembang aplikasi']],
 [74, 0.31747033786806195, ['kembang aplikasi']],
 [91, 0.31747033786806195, ['kembang aplikasi']],
 [92, 0.31747033786806195, ['kembang aplikasi']],
 [109, 0.31747033786806195, ['kembang aplikasi']],
 [110, 0.31747033786806195, ['kembang aplikasi']],
 [127, 0.31747033786806195, ['kembang aplikasi']],
 [128, 0.31747033786806195, ['kembang aplikasi']],
 [145, 0.31747033786806195, ['kembang aplikasi']],
 [146, 0.31747033786806195, ['kembang aplikasi']],
 [163, 0.31747033786806195, ['kembang aplikasi']],


In [25]:
for x in new_result[1:5]: 
    print('Result', x[0]+1) 
    print('QUERY', x[2]) 
    print(paper[x[0]-1][1]) 
    print(paper[x[0]-1][2][:200] + '...')
    print()

Result 2
QUERY ['kembang aplikasi']
RANCANG BANGUN APLIKASI KOMUNIKASI AUDIO PADA JARINGAN NIRKABEL LOKAL BERBASIS ANDROID MENGGUNAKAN ALGORITMA JOINT CODING RATE CONTROL
Perkembangan aplikasi Android dewasa ini sangat pesat aplikasi dikembangkan untuk menyelesaikan permasalahan diberbagai bidang. Saat ini ada beberapa kategori aplikasi Android yang dikembangkan sepert...

Result 3
QUERY ['kembang aplikasi']
DESIGN AND IMPLEMENTATION OF ANDROID BASED AUDIO COMMUNICATION OVER WIRELESS LOCAL NETWORK USING JOINT CODING RATE CONTROL ALGORITHM
Perkembangan aplikasi Android dewasa ini sangat pesat aplikasi dikembangkan untuk menyelesaikan permasalahan diberbagai bidang. Saat ini ada beberapa kategori aplikasi Android yang dikembangkan sepert...

Result 20
QUERY ['kembang aplikasi']
RANCANG BANGUN APLIKASI KOMUNIKASI AUDIO PADA JARINGAN NIRKABEL LOKAL BERBASIS ANDROID MENGGUNAKAN ALGORITMA JOINT CODING RATE CONTROL
Perkembangan aplikasi Android dewasa ini sangat pesat aplikasi dikembangkan un

In [26]:
idx_res = [x[0]-1 for x in new_result[1:]]
file = []
for i,x in enumerate(paper):
    if i in idx_res:
        file.append([x[1],x[2],'ok_ok'])
    else:
        file.append([x[1],x[2],''])
df = pd.DataFrame(file)
df.to_excel('hasil expansion ' +init_query+ '.xlsx', header=False, index=False)